In [1]:
from nltk import bigrams
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

import itertools
import random
import pickle
import pandas as pd

# WMT16 and WMT17 segment-level data 

In [2]:
wmt16 = pickle.load(open('../wmt16-19-metrics-shared-task/data/pickles/wmt16-seg_level-agg.pkl', 'rb'))
wmt16['year'] = len(wmt16)*[2016]

wmt17 = pickle.load(open('../wmt16-19-metrics-shared-task/data/pickles/wmt17-seg_level-agg.pkl', 'rb'))
wmt17['year'] = len(wmt17)*[2017]

wmt_mst_seg = pd.concat([wmt16, wmt17])

In [3]:
wmt_mst_seg.head(5)

,lp,system,sid,sentBLEU,score,output,reference,source,year
0,en-ru,jhu-pbmt,1092,0.273012,0.363122,43 закусочных нарушил требования к организации...,43 закусочных нарушили требования к устройству...,43 eateries violated requirements for the orga...,2016
1,en-ru,online-G,750,0.076668,-0.450232,"Учитывая, что он представляет собой сугубо пра...","Кажется маловероятным, что Кэмерон, будучи, по...","Given he is an avowedly hands-on parent, it se...",2016
2,en-ru,AFRL-MITLL-phrase-based,2786,0.252464,0.113451,Печать процессы глобализации должны помогли ум...,Распечатать Процессы глобализации должны были ...,Printing the Processes of Globalisation should...,2016
3,en-ru,LIMSI,250,0.531697,-0.257524,"Это нечто значительное, странное или необычное...","Что-то важное, странное или необычное происход...","Is something significant, bizarre or unusual h...",2016
4,en-ru,AFRL-MITLL-phrase-based,88,0.097414,-0.695001,"В то время как сами праздничные дни месяца, в ...","Хотя до праздников еще несколько месяцев, сезо...","While the holidays themselves are months away,...",2016


In [172]:
reference = word_tokenize(wmt_mst_seg[wmt_mst_seg.lp == 'de-en'].iloc[10]['reference'])
hypothesis = word_tokenize(wmt_mst_seg[wmt_mst_seg.lp == 'de-en'].iloc[10]['output'])
print(' '.join(reference))
print(' '.join(hypothesis))

It is about tolerance , acceptance and the vision of an open society .
It is a matter of tolerance , acceptance and the Vision of an open society .


# Level sets

### Permuting chunks defined by bigram mismatches

In [173]:
sentence_bleu([reference], hypothesis)

0.4935578819979932

In [174]:
def chunks(reference, hypothesis):
    s = set(bigrams(reference))
    
    match = []
    for idx, bg in enumerate(zip(hypothesis, hypothesis[1:])):
        match.append(bg in s)
    
    segments = [[hypothesis[0]]]
    
    for idx, b in enumerate(match[:-1]):
        if match[idx] == match[idx+1]:
            segments[-1].append(hypothesis[idx+1])
        else:
            if match[idx]:
                segments[-1].append(hypothesis[idx+1])
                segments.append([])
            else:
                segments.append([])
                segments[-1].append(hypothesis[idx+1])
    segments[-1].append(hypothesis[-1])
        
    return segments

def permute_chunks(segments):
    return sorted(segments, key=lambda x: random.random())

In [179]:
segments = chunks(reference, hypothesis)
print(segments)
adv_hypothesis = sum(permute_chunks(segments), [])
print()
print(' '.join(adv_hypothesis))

[['It', 'is'], ['a', 'matter', 'of'], ['tolerance', ',', 'acceptance', 'and', 'the'], ['Vision'], ['of', 'an', 'open', 'society', '.']]

Vision a matter of of an open society . It is tolerance , acceptance and the


In [180]:
sentence_bleu([reference], adv_hypothesis)

0.4935578819979932

### Substituting words that do not overlap

In [10]:
def overlapping(reference, hypothesis):
    overlap = []
    for i in hypothesis:
        overlap.append(i in reference)
    return overlap

def random_replace(reference, hypothesis, overlap, replace_prob=0.5, replacement_func=lambda x: 'WORD'):
    adv_hypothesis = hypothesis.copy()
    for i, ol in enumerate(overlap):
        if not ol:
            if random.random() < replace_prob:
                adv_hypothesis[i] = replacement_func(hypothesis[i])
    return adv_hypothesis

In [11]:
overlap = overlapping(reference, hypothesis)
adv_hypothesis = random_replace(reference, hypothesis, overlap, 1)
print(adv_hypothesis)

['A', 'good', 'prank', 'is', 'funny', ',', 'but', 'WORD', 'takes', 'WORD', 'moments', 'WORD', 'WORD', 'WORD', 'WORD', 'WORD', '.']


In [12]:
sentence_bleu([reference], adv_hypothesis)

0.3807134866446316